In [20]:
import pandas as pd
import numpy as np
path = 'US Census Edu Attain Pop 25 and over.csv'

# refer to columns dictionary text file for column label definitions
cols_list = ['GEO_ID','NAME','S1501_C01_019E', 'S1501_C01_019M', 'S1501_C01_020E', 
             'S1501_C01_020M', 'S1501_C01_021E', 'S1501_C01_021M'] 

# read CSV file
zips_df = pd.read_csv(path, usecols=cols_list)

# drop first row if column labels are desired
zips_df = zips_df.drop(0)

# # promote first row as headers if actual column names are desired
# new_header = zips_df.iloc[0] 
# zips_df = zips_df[1:] 
# zips_df.columns = new_header 

# replace all values marked by "*****" as 0; see Symbols Dictionary for definition
zips_df = zips_df.replace({'*****':0})

# change numeric columns to int32 data type
zips_df = zips_df.astype({col: 'int32' for col in zips_df.columns[2:]})

zips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33120 entries, 1 to 33120
Data columns (total 8 columns):
GEO_ID            33120 non-null object
NAME              33120 non-null object
S1501_C01_019E    33120 non-null int32
S1501_C01_019M    33120 non-null int32
S1501_C01_020E    33120 non-null int32
S1501_C01_020M    33120 non-null int32
S1501_C01_021E    33120 non-null int32
S1501_C01_021M    33120 non-null int32
dtypes: int32(6), object(2)
memory usage: 1.5+ MB


In [22]:
# Check error propagation calculation in dataframe where errSum = sqrt( err1**2 + err2**2) )

# add two child estimate columns (S1501_C01_020E and S1501_C01_021E) to check parent estimate column
zips_df['Estimate Check'] = zips_df['S1501_C01_020E'] + zips_df['S1501_C01_021E']

# add column of booleans to show if calculations are off by greater than 5%
zips_df['Estimate Match?'] = (abs(zips_df['S1501_C01_019E'] - zips_df['Estimate Check'])/zips_df['S1501_C01_019E']) < 0.05

# add two child Margin of Error (MoE) columns (S1501_C01_020M and S1501_C01_021M) to check parent Margin of 
# Error column by using error propagation formula where errSum = sqrt( err1**2 + err2**2)
zips_df['MoE Check'] = np.sqrt(np.square(zips_df['S1501_C01_020M']) + np.square(zips_df['S1501_C01_021M']))

# add column of booleans to show if calculations are off by greater than 5%
zips_df['MoE Match?'] = (abs(zips_df['S1501_C01_019M'] - zips_df['MoE Check'])/zips_df['S1501_C01_019M']) < 0.05

zips_df

,GEO_ID,NAME,S1501_C01_019E,S1501_C01_019M,S1501_C01_020E,S1501_C01_020M,S1501_C01_021E,S1501_C01_021M,Estimate Check,Estimate Match?,MoE Check,MoE Match?
1,8600000US00601,ZCTA5 00601,1938,120,1389,146,416,138,1805,False,200.897984,False
2,8600000US00602,ZCTA5 00602,4806,4,3569,260,1155,206,4724,True,331.716747,False
3,8600000US00603,ZCTA5 00603,6285,205,5183,243,1875,264,7058,False,358.810535,False
4,8600000US00606,ZCTA5 00606,860,54,576,117,96,85,672,False,144.616735,False
5,8600000US00610,ZCTA5 00610,3473,80,2761,224,764,231,3525,True,321.771658,False
...,...,...,...,...,...,...,...,...,...,...,...,...
33116,8600000US99923,ZCTA5 99923,0,9,0,9,0,9,0,False,12.727922,False
33117,8600000US99925,ZCTA5 99925,88,20,86,19,15,10,101,False,21.470911,False
33118,8600000US99926,ZCTA5 99926,158,35,141,33,11,10,152,True,34.481879,True
33119,8600000US99927,ZCTA5 99927,0,9,0,9,0,9,0,False,12.727922,False
